<a href="https://colab.research.google.com/github/chchang1990/rs_pm25/blob/dev/rs_pm25_gee_data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Remote sensing application in estimating ground-level PM2.5 concentration**

# Package installation and import

In [ ]:
!pip install earthengine-api \
             geemap \
             xee

In [ ]:
import ee
import geemap

import xarray as xr

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Authenticate and initialize the Google Earth Engine (GEE) API

Note: It requires registrating Google Earth Engine (https://earthengine.google.com/signup/) and enabling it in your Google Cloud Project (https://github.com/googlecolab/colabtools/issues/4228)

In [ ]:
ee.Authenticate()
ee.Initialize(project='gcstorage-378004', opt_url='https://earthengine-highvolume.googleapis.com')

# Define Area-Of-Interest (AOI) and retrieve data from the GEE

Define AOI as Harris county, Texas. The shapefile can be obtained from the GEE as well.

In [ ]:
aoi = (ee.FeatureCollection("TIGER/2018/Counties")
       .filter(ee.Filter.eq('NAME', 'Harris'))
       .filter(ee.Filter.eq('STATEFP','48'))
       # Aparently there are more than one Harris counties in the U.S., hence refine the query to be in Texas
       # (The State FIPS code for Texas is 48)
).geometry()

aoi_lon_lat = np.asarray(aoi.getInfo()['coordinates'][0])
#aoi_lon_lat = np.asarray(aoi.getInfo()['features'][0]['geometry']['coordinates'][0]

aoi_clon, aoi_clat = np.nanmean(aoi_lon_lat, axis=0)

map_aoi = geemap.Map(center=(aoi_clat, aoi_clon), zoom=10)
map_aoi.addLayer(aoi)
map_aoi


Data retrieval

In [ ]:
def clip_map(img):
    return img.clip(aoi)

#def scaling_map(img):
#    return img.multiply(ee.Image(0.001))

In [ ]:
# ----- MODIS MAIAC AOD product -----
leg1 = ee.Geometry.Rectangle(113.33, -43.63, 153.56, -10.66)
leg2 = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

aod_col = (ee.ImageCollection('MODIS/061/MCD19A2_GRANULES')
                     .select('Optical_Depth_055')
                     .filterDate('2022-07-10', '2022-07-12')
).map(clip_map)
# If applying scale here, somehow it will take forever for processing the data after converting it to XARRAY



In [ ]:
# ----- Visualization color table -----
vis_aod = {
  'min': 0,
  'max': 700, # AOD in Texas is usually from 0.01 to 0.68 (Ghahremanloo et al., 2021)
  'palette': ['purple', 'blue', 'cyan', 'green', 'yellow', 'red']
};


map_vars = geemap.Map(center=(aoi_clat, aoi_clon), zoom=10)
map_vars.addLayer(aod_col.max(), vis_aod, 'Example AOD in 2022')
map_vars.addLayerControl()
map_vars

Convert the data from server-side to client side as XARRAY. Data will be clipped to the bounding box that encompasses the pre-defined AOI.

It seems like the XEE package doesn't support clipping to irregular shape yet. Will need to clip it with Geopandas after exporting Harris county, TX boundary as .SHP

In [ ]:
aod_da = xr.open_dataset(
    aod_col,
    engine='ee',
    projection='EPSG:4326', # Reproject to EPSG:4326 (WGS84 lat, lon)
    geometry=aoi,
    scale=0.01 # Spatial resolution. Its unit depends on the projection.
).transpose('time','lat','lon')['Optical_Depth_055']*0.001 # Get the AOD variable and scale it

In [ ]:
plt.imshow(aod_da.max(dim='time', skipna=True))
plt.colorbar()
plt.show()